In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [1]:
import os
def get_image_files(folder_path):
    image_extensions = ['.jpeg','.jpg']

    image_files = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            if any(file.lower().endswith(ext) for ext in image_extensions):
                image_files.append(file)

    return image_files


import json

images=get_image_files('/home/coder/Abid/proj_with_marco/Data/train_data')
with open('all_caps.json','r') as f:
    caps=json.load(f)

labels={}
for k,v in zip(caps.keys(),caps.values()):
    labels[k]=v[-1]

In [3]:
prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a vision assistant, given an image you describe it briefly.<|eot_id|><|start_header_id|>user<|end_header_id|><|image|>What is the person doing in this image?<|eot_id|><|start_header_id|>assistant<|end_header_id|>description:"""

In [4]:
data=[("/home/coder/Abid/proj_with_marco/Data/train_data/"+img,prompt,caps[img.split('_m',1)[0].replace('.','_')][-1]+" <|end_of_text|>") for img in images[:]]
data

[('/home/coder/Abid/proj_with_marco/Data/train_data/Walk_p04_r00_v05_c05_middle.jpg',
  '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a vision assistant, given an image you describe it briefly.<|eot_id|><|start_header_id|>user<|end_header_id|><|image|>What is the person doing in this image?<|eot_id|><|start_header_id|>assistant<|end_header_id|>description:',
  'The person is walking into the living room. <|end_of_text|>'),
 ('/home/coder/Abid/proj_with_marco/Data/train_data/Sitdown_p12_r01_v03_c04_middle.jpg',
  '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a vision assistant, given an image you describe it briefly.<|eot_id|><|start_header_id|>user<|end_header_id|><|image|>What is the person doing in this image?<|eot_id|><|start_header_id|>assistant<|end_header_id|>description:',
  'The person is sitting on the couch. <|end_of_text|>'),
 ('/home/coder/Abid/proj_with_marco/Data/train_data/Walk_p10_r05_v13_c07_middle.jpg',
  '<|begin_of

##  Fine-Tuning


In [5]:

import torch
from PIL import Image
from transformers import AutoModelForImageTextToText, AutoProcessor
import requests
from transformers import AutoTokenizer
import torch
from torch.optim import AdamW  # ✅ Correct and recommended
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
# Check if CUDA is available
from transformers import  AutoTokenizer


# device = "cuda:1" if torch.cuda.is_available() else "cpu"
device = "cuda" if torch.cuda.is_available() else "cpu"



model_id = "meta-llama/Llama-3.2-11B-Vision-Instruct"
base_model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16
).to(device)
processor = AutoProcessor.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# === LoRA setup ===
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(base_model, lora_config)
model.train()



/home/coder/miniconda3/envs/gouga/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:00<00:00, 27.67it/s]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MllamaForConditionalGeneration(
      (vision_model): MllamaVisionModel(
        (patch_embedding): Conv2d(3, 1280, kernel_size=(14, 14), stride=(14, 14), padding=valid, bias=False)
        (gated_positional_embedding): MllamaPrecomputedPositionEmbedding(
          (tile_embedding): Embedding(9, 8197120)
        )
        (pre_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (post_tile_positional_embedding): MllamaPrecomputedAspectRatioEmbedding(
          (embedding): Embedding(9, 5120)
        )
        (layernorm_pre): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (layernorm_post): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (transformer): MllamaVisionEncoder(
          (layers): ModuleList(
            (0-31): 32 x MllamaVisionEncoderLayer(
              (self_attn): MllamaVisionSdpaAttention(
               

In [6]:

from torch.utils.data import Dataset, DataLoader

# === Configuration ===
epochs = 7
lr = 5e-5
batch_size = 1


class FTDataset(Dataset):
    def __init__(self, data_list, processor):
        """
        data_list: list of (image_path, prompt, label)
        processor: Hugging Face processor (tokenizer + image preprocessor)
        """
        self.data = data_list
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, prompt, label = self.data[idx]
        image = Image.open(image_path).convert("RGB")

        inputs = self.processor(text=prompt, images=image, return_tensors="pt", padding=True)
        label_ids = self.processor.tokenizer(label, return_tensors="pt").input_ids[0]

        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "pixel_values": inputs["pixel_values"].squeeze(0),
            "label_ids": label_ids
        }



# === Init Dataloader ===
dataset = FTDataset(data,processor=processor)
dataloader = DataLoader(dataset, batch_size=batch_size)


In [7]:


# === Optimizer ===
optimizer = AdamW(model.parameters(), lr=lr)

# === Training loop ===
for epoch in range(epochs):
    for batch in dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        pixel_values = batch["pixel_values"].to(device)
        label_ids = batch["label_ids"].to(device)

        full_input_ids = torch.cat([input_ids, label_ids], dim=1)
        full_attention_mask = torch.cat([attention_mask, torch.ones_like(label_ids)], dim=1)

        # Mask labels: only compute loss on label portion
        labels = torch.full_like(full_input_ids, -100)
        labels[:, input_ids.shape[1]:] = label_ids

        outputs = model(
            input_ids=full_input_ids,
            attention_mask=full_attention_mask, 
            labels=labels
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        print(f"Epoch {epoch+1} - Loss: {loss.item():.4f}")



`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch 1 - Loss: 7.0675
Epoch 1 - Loss: 6.5481
Epoch 1 - Loss: 6.1351
Epoch 1 - Loss: 6.1375
Epoch 1 - Loss: 5.9309
Epoch 1 - Loss: 4.9899
Epoch 1 - Loss: 5.6071
Epoch 1 - Loss: 5.2417
Epoch 1 - Loss: 5.1329
Epoch 1 - Loss: 5.2329
Epoch 1 - Loss: 4.3001
Epoch 1 - Loss: 5.0009
Epoch 1 - Loss: 5.7564
Epoch 1 - Loss: 5.1470
Epoch 1 - Loss: 6.1195
Epoch 1 - Loss: 4.5412
Epoch 1 - Loss: 3.9975
Epoch 1 - Loss: 3.3926
Epoch 1 - Loss: 4.5545
Epoch 1 - Loss: 4.5010
Epoch 1 - Loss: 4.5449
Epoch 1 - Loss: 3.9828
Epoch 1 - Loss: 3.1715
Epoch 1 - Loss: 4.0016
Epoch 1 - Loss: 3.8669
Epoch 1 - Loss: 3.5682
Epoch 1 - Loss: 3.6800
Epoch 1 - Loss: 2.7044
Epoch 1 - Loss: 3.0204
Epoch 1 - Loss: 3.2493
Epoch 1 - Loss: 2.3105
Epoch 1 - Loss: 3.2381
Epoch 1 - Loss: 2.8517
Epoch 1 - Loss: 1.8631
Epoch 1 - Loss: 2.2381
Epoch 1 - Loss: 2.0867
Epoch 1 - Loss: 1.8958
Epoch 1 - Loss: 1.8540
Epoch 1 - Loss: 1.1345
Epoch 1 - Loss: 1.9422
Epoch 1 - Loss: 1.7818
Epoch 1 - Loss: 1.0328
Epoch 1 - Loss: 2.5548
Epoch 1 - L

In [6]:
model.eval()
generated=[]
"Abid/proj_with_marco/Data/train_data"
test_data=get_image_files("/home/coder/Abid/proj_with_marco/Data/train_data")
for image in test_data:
    pil_image = Image.open("/home/coder/Abid/proj_with_marco/Data/train_data/"+image).convert("RGB")

    # Define your text prompt
    #prompt ="<|image|>what is the person doing?"

    with torch.no_grad():
        
        inputs = processor(text=prompt, images=pil_image, return_tensors="pt").to(device) 
        outputs=model.generate(
            **inputs,
            max_new_tokens= 20,
            min_length= 20,
            do_sample= True,
            no_repeat_ngram_size = 4
        )
        generated_text = processor.decode(outputs[0], skip_special_tokens=False)
    generated.append(f"{image} -- {generated_text.split('description:')[1]}")
    print(f"{image} -- {generated_text.split('description:')[1]}")
    
with open('noFinetune.json','w') as f:
    json.dump(generated,f)


Walk_p04_r00_v05_c05_middle.jpg --  The person is standing in the living room, holding a Wii remote and playing a video game.<|eot_id|>
Sitdown_p12_r01_v03_c04_middle.jpg --  The person in the image is sitting on a green couch, holding a white remote control in their hands
Walk_p10_r05_v13_c07_middle.jpg --  The image is blurry, but it appears to show a person standing at a counter. The person is
Sitdown_p09_r03_v07_c04_middle.jpg --  The person in this image is sitting on a green couch, possibly reading or looking down at a book
Sitdown_p10_r04_v03_c04_middle.jpg --  The person in the image appears to be sitting on a green couch, leaning forward with their hands on
Drink.Frombottle_p06_r01_v10_c02_middle.jpg --  the person in the image appears to be sitting at a table with a white tablecloth, eating a
Laydown_p03_r00_v19_c05_middle.jpg --  The person in the image is sitting on a green couch, watching a football game on the television.
Walk_p04_r02_v07_c05_middle.jpg --  The person in 

In [9]:
import os
from PIL import Image
import pprint

def read_img_folder(folder='../Data/train_data',suffix='_middle.jpg'):
    imgs = [img.replace(suffix,'') for img in os.listdir(folder)]
    print(f"{len(imgs)} Images, from idx 0 to {len(imgs)-1}")
    return imgs

def show_img_and_caps(imgs,folder='../Data/train_data',suffix='_middle.jpg',idx=0):
    print(imgs[idx])
    pic = Image.open(folder+"/"+imgs[idx]+suffix)
    pic.show()
    pprint.pprint(caps[imgs[idx]])
imgs = read_img_folder()

130 Images, from idx 0 to 129


In [16]:
# show_img_and_caps(imgs,idx=6)
caps['Drink_Frombottle_p03_r01_v03_c04']

['Drink_Frombottle',
 'The man is sitting on the sofa and drinking from a glass. \n    The man is sitting on the sofa and drinking from a glass. \n    The man is sitting on the sofa and drinking from a glass. \n    The man is sitting on',
 'The person is sitting on the couch and drinking from a cup.',
 'the person is drinking water from a bottle',
 'The person is sitting on the person is drinking.',
 'The person is sitting on a couch and drinking from a bottle.']

## 1 image overfit

In [4]:
# === Single example ===
image = Image.open("tv.jpg").convert("RGB")
prompt = "what is the person doing?"
label = "the person is watching tv <|end_of_text|> "

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)  # ✅ correct
target_ids = processor.tokenizer(label, return_tensors="pt").input_ids.to(device)

# Combine input + target
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

full_input_ids = torch.cat([input_ids, target_ids], dim=1)
full_attention_mask = torch.cat([attention_mask, torch.ones_like(target_ids)], dim=1)

# Mask loss: only compute over label
labels = torch.full_like(full_input_ids, -100)
labels[:, input_ids.shape[1]:] = target_ids

# === Optimizer ===
optimizer = AdamW(model.parameters(), lr=5e-3)

# === Overfit Loop ===
for step in range(100):
    outputs = model(
        input_ids=full_input_ids,
        attention_mask=full_attention_mask,
        labels=labels
    )
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    print(f"Step {step} - Loss: {loss.item():.4f}")

    # Stop early if loss is very low
    if loss.item() < 0.01:
        print("✅ Overfitting successful.")
        break


image_path = "tv.jpg"  # Replace with your actual image path
image = Image.open(image_path).convert("RGB")

# Define your text prompt
prompt ="<|image|> what is the person doing ?"

with torch.no_grad():
    
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device) 
    outputs=model.generate(**inputs,max_length=30)
    generated_text = processor.decode(outputs[0], skip_special_tokens=False)

print("Generated Text:", generated_text)


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step 0 - Loss: 10.3623
Step 1 - Loss: 4.4733
Step 2 - Loss: 7.8711
Step 3 - Loss: 5.2411
Step 4 - Loss: 6.9303
Step 5 - Loss: 6.3600
Step 6 - Loss: 28.7334
Step 7 - Loss: 10.7192
Step 8 - Loss: 7.8417
Step 9 - Loss: 6.2502
Step 10 - Loss: 6.5285
Step 11 - Loss: 4.9882
Step 12 - Loss: 4.6794
Step 13 - Loss: 4.5411
Step 14 - Loss: 3.7174
Step 15 - Loss: 24.9111
Step 16 - Loss: 3.7913
Step 17 - Loss: 3.8505
Step 18 - Loss: 4.0146
Step 19 - Loss: 3.7713
Step 20 - Loss: 3.5547
Step 21 - Loss: 3.4515
Step 22 - Loss: 3.1453
Step 23 - Loss: 2.9508
Step 24 - Loss: 2.8847
Step 25 - Loss: 2.7766
Step 26 - Loss: 2.6599
Step 27 - Loss: 2.4898
Step 28 - Loss: 2.3936
Step 29 - Loss: 2.3410
Step 30 - Loss: 2.0899
Step 31 - Loss: 2.0051
Step 32 - Loss: 1.9911
Step 33 - Loss: 1.9856
Step 34 - Loss: 1.9323
Step 35 - Loss: 1.9026
Step 36 - Loss: 1.8333
Step 37 - Loss: 1.7020
Step 38 - Loss: 1.6065
Step 39 - Loss: 1.5376
Step 40 - Loss: 1.4488
Step 41 - Loss: 1.3601
Step 42 - Loss: 1.2625
Step 43 - Loss: 1